In [5]:
import time

from osm_runner import Runner  # pip install osm-runner
import pandas as pd

from arcgis.features import FeatureLayer, GeoAccessor, GeoSeriesAccessor
from arcgis.geoenrichment import enrich
from arcgis import dissolve_boundaries
from arcgis.geometry import project
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, GeoAccessor, GeoSeriesAccessor
from arcgis.geoenrichment import enrich
from arcgis import dissolve_boundaries
from arcgis.geometry import project
from arcgis.gis import GIS

In [6]:
def str_to_cord(x):
    lat,long = float(x.split(",")[0]),float(x.split(",")[1])
    return lat,long

# Community center

In [7]:
import requests
import json
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="community_centre"](area);
 way["amenity"="community_centre"](area);
 rel["amenity"="community_centre"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
community_centre_data = response.json()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Collect coords into list
coords = []
for element in community_centre_data['elements']:
    if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        coords.append((lon, lat))
    elif 'center' in element:
        lon = element['center']['lon']
        lat = element['center']['lat']
        coords.append((lon, lat))
# Convert coordinates into numpy array
X = np.array(coords)
plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('community_centre in Germany')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

In [5]:
import pandas as pd
df = pd.DataFrame()

In [6]:
df["lat"] = X[:, 1]
df["long"] =X[:, 0]
df["dummy"] = df.lat.apply(lambda x: str(round(x,5)))
df["dummy1"] = df.long.apply(lambda x: "," +str(round(x,5)))
df["dummy2"] = df.dummy+df.dummy1
df["cord"] = df.dummy2.apply(lambda x: str_to_cord(x))
df.drop(["dummy","dummy1","dummy2"],axis =1,inplace=True)

In [7]:
df.head()

,lat,long,cord
0,49.663964,8.446186,"(49.66396, 8.44619)"
1,49.034349,8.371743,"(49.03435, 8.37174)"
2,48.819820,9.419496,"(48.81982, 9.4195)"
3,52.274467,8.084950,"(52.27447, 8.08495)"
4,52.285763,10.455139,"(52.28576, 10.45514)"


In [8]:
dn =pd.read_pickle("d_after_biergarten_OSM.pickle")
#dn= dn[["city","latitude","longitude","park_id","postal_code"]]
dn["dummy"] = dn.latitude.apply(lambda x: str(round(x,5)))
dn["dummy1"] = dn.longitude.apply(lambda x: "," +str(round(x,5)))
dn["dummy2"] = dn.dummy+dn.dummy1
dn["cord"] = dn.dummy2.apply(lambda x: str_to_cord(x))
dn.drop(["dummy","dummy1","dummy2"],axis =1,inplace=True)

In [9]:
dn.head()

,park_id,clc_code,area_code,evse_id,datetime_start,datetime_stop,Charging_Started,Charging_Stop,city,latitude,...,day_of_week_name_Start,Precipitation,Temperature,pd,Count_Charging,Charging_time_in_hours_log,cord,Nearest_Bahnhof_km,Nearest_Cafe_km,Nearest_Biergarten_km
0,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:00:13.058,1.0,0.0,Kremmen,52.7596,...,Tuesday,0.0,9.566667,158.737964,58,3.178042,"(52.7596, 13.0239)",7.285195,0.850239,13.084341
1,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:01:16.290,1.0,0.0,Kremmen,52.7596,...,Tuesday,0.0,9.566667,158.737964,58,-4.061555,"(52.7596, 13.0239)",7.285195,0.850239,13.084341
2,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:58:30.703,1.0,0.0,Kremmen,52.7596,...,Tuesday,0.0,9.566667,158.737964,58,-0.029028,"(52.7596, 13.0239)",7.285195,0.850239,13.084341
3,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:58:28.352,2019-12-17 09:00:13.058,1.0,0.0,Kremmen,52.7596,...,Tuesday,0.0,9.566667,158.737964,58,3.136749,"(52.7596, 13.0239)",7.285195,0.850239,13.084341
4,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:58:28.352,2019-12-17 09:01:16.290,1.0,0.0,Kremmen,52.7596,...,Tuesday,0.0,9.566667,158.737964,58,3.137509,"(52.7596, 13.0239)",7.285195,0.850239,13.084341


# Process 

In [10]:
def  filter_lat_long(cord,th=0.1):
    la1,ln1 = cord
    dk = df.copy()
    return(
    dk
        [dk["lat"]<=la1+th]
        [dk["lat"]>=la1-th]
        [dk["long"]<=ln1+th]
        [dk["long"]>=ln1-th])

    

In [11]:
# 0.1 latitude is 11 km square
filter_lat_long(dn.cord.iloc[0],th=0.2)

<ipython-input-10-3fdbe1c92f06>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dk


,lat,long,cord
390,52.564413,13.094358,"(52.56441, 13.09436)"
1230,52.571304,13.213562,"(52.5713, 13.21356)"
1401,52.561725,13.209305,"(52.56172, 13.20931)"
2616,52.564906,13.090612,"(52.56491, 13.09061)"
2676,52.727655,12.991048,"(52.72766, 12.99105)"


In [12]:
import geopy
from geopy import distance
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(distance.distance(coords_1, coords_2).km )


279.35290160430094


In [13]:
def nearest(c,th=0.1):
    dk = filter_lat_long(cord = c,th=th)
    if(len(dk)>0):
        dk["distance"] = dk.cord.apply(lambda x: distance.distance(c,x))
        dk = dk.sort_values("distance",ascending = True)
        v  = dk.distance.iloc[0].km
    else:
        v = 1000
    return v

In [14]:
nearest(dn.cord.iloc[0],th=0.2)

<ipython-input-10-3fdbe1c92f06>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dk


4.189867991855165

In [15]:
dn["Nearest_Community_Centre_km"] = dn.cord.apply(lambda x :nearest(x,th=0.2))

<ipython-input-10-3fdbe1c92f06>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dk


In [16]:
dn

,park_id,clc_code,area_code,evse_id,datetime_start,datetime_stop,Charging_Started,Charging_Stop,city,latitude,...,Precipitation,Temperature,pd,Count_Charging,Charging_time_in_hours_log,cord,Nearest_Bahnhof_km,Nearest_Cafe_km,Nearest_Biergarten_km,Nearest_Community_Centre_km
0,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:00:13.058,1.0,0.0,Kremmen,52.7596,...,0.0,9.566667,158.737964,58,3.178042,"(52.7596, 13.0239)",7.285195,0.850239,13.084341,4.189868
1,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:01:16.290,1.0,0.0,Kremmen,52.7596,...,0.0,9.566667,158.737964,58,-4.061555,"(52.7596, 13.0239)",7.285195,0.850239,13.084341,4.189868
2,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:00:13.369,2019-12-17 09:58:30.703,1.0,0.0,Kremmen,52.7596,...,0.0,9.566667,158.737964,58,-0.029028,"(52.7596, 13.0239)",7.285195,0.850239,13.084341,4.189868
3,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:58:28.352,2019-12-17 09:00:13.058,1.0,0.0,Kremmen,52.7596,...,0.0,9.566667,158.737964,58,3.136749,"(52.7596, 13.0239)",7.285195,0.850239,13.084341,4.189868
4,hj_31c455b8-420e-46af-bce6-436b65ccc416,Suburban,120650165165,hj_DE*BLN*E*174060033,2019-12-17 09:58:28.352,2019-12-17 09:01:16.290,1.0,0.0,Kremmen,52.7596,...,0.0,9.566667,158.737964,58,3.137509,"(52.7596, 13.0239)",7.285195,0.850239,13.084341,4.189868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940929,sl_23897386,Urban,051120000000,sl_23901984,2021-08-07 19:03:56.000,2021-08-07 16:02:00.000,1.0,0.0,Niestetal,51.3144,...,0.0,8.033333,2142.121993,331,3.042987,"(51.3144, 9.5549)",6.690153,0.948615,1.871055,2.124286
1940930,sl_23897386,Urban,051120000000,sl_23901984,2021-08-07 19:03:56.000,2021-08-07 16:28:29.000,1.0,0.0,Niestetal,51.3144,...,0.0,8.033333,2142.121993,331,3.063819,"(51.3144, 9.5549)",6.690153,0.948615,1.871055,2.124286
1940931,sl_23897386,Urban,051120000000,sl_23901984,2021-08-07 19:03:56.000,2021-08-07 19:55:30.000,1.0,0.0,Niestetal,51.3144,...,0.0,8.033333,2142.121993,331,-0.151469,"(51.3144, 9.5549)",6.690153,0.948615,1.871055,2.124286
1940932,sl_23897386,Urban,051120000000,sl_23901984,2021-08-10 15:29:47.000,2021-08-10 16:57:44.000,1.0,0.0,Niestetal,51.3144,...,0.0,8.033333,2142.121993,331,0.382424,"(51.3144, 9.5549)",6.690153,0.948615,1.871055,2.124286


In [17]:
dn.to_pickle("d_after_Community_Centre_OSM.pickle")

In [18]:
dn.to_csv("d_after_Community_Centre_OSM.csv")